# Run TFlite inference

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tflite_runtime.interpreter as tflite


ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)


In [6]:


interpreter = tflite.Interpreter("/home/lananh/GISLR/model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

train = pd.read_csv('/home/lananh/GISLR/train.csv')
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()
# output = prediction_fn(inputs=demo_raw_data)
# sign = output["outputs"].argmax()

# Function to predict sign based on Mediapipe parquet

In [7]:
def get_prediction(prediction_fn, pd_file):
    xyz_np = load_relevant_data_subset(pd_file)
    prediction = prediction_fn(inputs=xyz_np)
    pred = prediction["outputs"].argmax()
    sign = ORD2SIGN[pred]
    pred_conf = prediction["outputs"][pred]
    print(f'PREDICTED SIGN: {sign} [{pred}], CONFIDENCE: {pred_conf:0.4}')


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/bird-0.parquet')

PREDICTED SIGN: bird [23], CONFIDENCE: 0.2142


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/duck-0.parquet')

PREDICTED SIGN: duck [67], CONFIDENCE: 0.6034


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/donkey-0.parquet')

# Take 3 sign from CHAT!!!

In [ ]:
train['sign'].unique()

array(['blow', 'wait', 'cloud', 'bird', 'owie', 'duck', 'minemy', 'lips',
       'flower', 'time', 'vacuum', 'apple', 'puzzle', 'mitten', 'there',
       'dry', 'shirt', 'owl', 'yellow', 'not', 'zipper', 'clean',
       'closet', 'quiet', 'have', 'brother', 'clown', 'cheek', 'cute',
       'store', 'shoe', 'wet', 'see', 'empty', 'fall', 'balloon',
       'frenchfries', 'finger', 'same', 'cry', 'hungry', 'orange', 'milk',
       'go', 'drawer', 'TV', 'another', 'giraffe', 'wake', 'bee', 'bad',
       'can', 'say', 'callonphone', 'finish', 'old', 'backyard', 'sick',
       'look', 'that', 'black', 'yourself', 'open', 'alligator', 'moon',
       'find', 'pizza', 'shhh', 'fast', 'jacket', 'scissors', 'now',
       'man', 'sticky', 'jump', 'sleep', 'sun', 'first', 'grass', 'uncle',
       'fish', 'cowboy', 'snow', 'dryer', 'green', 'bug', 'nap', 'feet',
       'yucky', 'morning', 'sad', 'face', 'penny', 'gift', 'night',
       'hair', 'who', 'think', 'brown', 'mad', 'bed', 'drink', 'stay',


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/police-0.parquet')

PREDICTED SIGN: police [174], CONFIDENCE: 0.2388


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/zebra-0.parquet')

PREDICTED SIGN: zebra [248], CONFIDENCE: 0.03786


In [ ]:
get_prediction(prediction_fn, '/home/lananh/GISLR/data_parquet/donkey-0.parquet')

PREDICTED SIGN: donkey [60], CONFIDENCE: 0.04517
